In [3]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17483, done.
remote: Total 17483 (delta 0), reused 0 (delta 0), pack-reused 17483 (from 1)
Receiving objects: 100% (17483/17483), 16.58 MiB | 16.80 MiB/s, done.
Resolving deltas: 100% (11991/11991), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━

In [4]:
!pip install torchvision matplotlib

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from google.colab import drive


In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!pip install tqdm
import zipfile
from tqdm import tqdm
import os

zip_path = '/content/drive/MyDrive/frames.zip'
extract_path = '/content/frames2'

# Mở file zip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    file_list = zip_ref.namelist()                 # Lấy danh sách file trong zip
    print(f"Tổng số file cần giải nén: {len(file_list)}")

    # Tạo thư mục đích nếu chưa có
    os.makedirs(extract_path, exist_ok=True)

    # Giải nén từng file với tqdm hiển thị tiến trình
    for file in tqdm(file_list, desc="Đang giải nén", unit="file"):
        zip_ref.extract(member=file, path=extract_path)

print("Giải nén hoàn tất!")


Tổng số file cần giải nén: 44220


Đang giải nén: 100%|██████████| 44220/44220 [02:16<00:00, 324.37file/s]

Giải nén hoàn tất!


In [7]:
# Kiểm tra nội dung giải nén
import os
os.listdir(extract_path)


['frames']

In [8]:
data_dir = '/content/frames2'

transform = transforms.Compose([
    transforms.Resize((128, 128)),  # resize ảnh cho CNN xử lý dễ hơn
    transforms.ToTensor()
])

dataset = datasets.ImageFolder(data_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [9]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [16]:
# Kiểm tra device
print("CUDA available:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

CUDA available: False
Device: CPU


In [10]:
#Thiết lập mô hình CNN
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # conv1
        x = self.pool(F.relu(self.conv2(x)))  # conv2
        x = self.pool(F.relu(self.conv3(x)))  # conv3
        x = x.view(x.size(0), -1)             # flatten
        x = F.relu(self.fc1(x))               # fc1
        x = self.fc2(x)                       # fc2
        return x

model = CNNModel()
print(model)

CNNModel(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=16384, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)


In [36]:
# Dataset
from PIL import Image # Added import

class DeepfakeDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.transform = transform
        self.image_paths = []
        self.labels = []
        print(f"Scanning directory: {root_dir}")
        for label_type, label in [('real', 0), ('fake', 1)]:
            class_dir = os.path.join(root_dir, label_type)
            print(f"Checking {class_dir}...")
            if not os.path.exists(class_dir):
                print(f"Warning: {class_dir} does not exist!")
                continue
            for subdir, _, files in os.walk(class_dir):
                for file in files:
                    if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                        self.image_paths.append(os.path.join(subdir, file))
                        self.labels.append(label)
        print(f"Found {len(self.image_paths)} images")

        self.transform = transforms.Compose([
            transforms.Resize((64, 64)),  # Giảm kích thước để chạy nhanh trên CPU
            transforms.ToTensor(),
        ])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        try:
            img_path = self.image_paths[idx]
            label = self.labels[idx]
            image = Image.open(img_path).convert("RGB")
            if self.transform:
                image = self.transform(image)
            print(f"Loaded image {img_path}: shape {image.shape}, label {label}")
            return image, label
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            return None, None

In [35]:
# Model
class CNNModel(nn.Module):
    def __init__(self, dropout_rate=0.5): # Added dropout_rate argument
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 32 * 32, 2)  # Điều chỉnh cho kích thước 64x64
        self.dropout = nn.Dropout(dropout_rate) # Added dropout layer

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(x.size(0), -1)
        x = self.dropout(x) # Added dropout to forward pass
        x = self.fc1(x)
        return x

In [40]:
from PIL import Image
img_path = "/content/frames2/frames/real/sample.jpg"  # Thay bằng đường dẫn ảnh thực tế
try:
    img = Image.open(img_path).convert("RGB")
    print("Ảnh hợp lệ")
except Exception as e:
    print(f"Ảnh lỗi: {e}")

Ảnh lỗi: [Errno 2] No such file or directory: '/content/frames2/frames/real/sample.jpg'


In [37]:
# Test
import sys # Added import for sys
from torch.utils.data.dataloader import default_collate # Added import for default_collate

data_dir = "/content/frames2/frames"
print("Thư mục tồn tại:", os.path.exists(data_dir))
print("Subfolders:", os.listdir(data_dir))

try:
    dataset = DeepfakeDataset(data_dir)
    print("Dataset size:", len(dataset))
    if len(dataset) == 0:
        raise ValueError("Dataset rỗng! Kiểm tra thư mục real/ và fake/ có ảnh không.")

    # Filter out None values from the dataset
    dataset = [data for data in dataset if data[0] is not None]
    print("Dataset size after filtering:", len(dataset))


    image, label = dataset[0]
    print("Sample image shape:", image.shape, "Label:", label)

    device = torch.device("cpu")  # Dùng CPU vì CUDA không khả dụng
    model = CNNModel().to(device)
    print("Model initialized")

    loader = DataLoader(dataset, batch_size=8, shuffle=True) # Removed collate_fn=collate_fn as filtering is done directly on the dataset
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    model.train()
    for epoch in range(1):  # Chỉ 1 epoch để test
        for images, labels in tqdm(loader, desc="Training"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            print(f"Loss: {loss.item()}", flush=True)
            sys.stdout.flush()  # Đảm bảo output hiển thị
            break  # Chỉ chạy 1 batch để test
except Exception as e:
    print(f"Error: {e}")

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
Loaded image /content/frames2/frames/fake/610_692/610_692_frame_11.jpg: shape torch.Size([3, 64, 64]), label 1
Loaded image /content/frames2/frames/fake/610_692/610_692_frame_6.jpg: shape torch.Size([3, 64, 64]), label 1
Loaded image /content/frames2/frames/fake/225_151/225_151_frame_0.jpg: shape torch.Size([3, 64, 64]), label 1
Loaded image /content/frames2/frames/fake/225_151/225_151_frame_5.jpg: shape torch.Size([3, 64, 64]), label 1
Loaded image /content/frames2/frames/fake/225_151/225_151_frame_9.jpg: shape torch.Size([3, 64, 64]), label 1
Loaded image /content/frames2/frames/fake/225_151/225_151_frame_8.jpg: shape torch.Size([3, 64, 64]), label 1
Loaded image /content/frames2/frames/fake/225_151/225_151_frame_10.jpg: shape torch.Size([3, 64, 64]), label 1
Loaded image /content/frames2/frames/fake/225_151/225_151_frame_6.jpg: shape torch.Size([3, 64, 64]), label 1
Loaded image /content/frames2/frames/fake/225_151/225_151_fra

In [38]:
# 2. CNN Model với Hook để theo dõi
class CNNModel(nn.Module):
    def __init__(self, dropout_rate=0.5):
        super(CNNModel, self).__init__()
        # Define layers
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)  # Thêm BatchNorm để ổn định
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(64 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 2)

        # Đếm hidden layers
        self.hidden_layers = [self.conv1, self.conv2, self.conv3, self.fc1]
        print(f"Number of hidden layers: {len(self.hidden_layers)}")

        # Dictionary để lưu activation và gradient
        self.activations = {}
        self.gradients = {}

        # Đăng ký hook
        self._register_hooks()

    def _register_hooks(self):
        def hook_fn(name):
            def hook(module, input, output):
                self.activations[name] = output.detach()
            def grad_hook(module, grad_input, grad_output):
                self.gradients[name] = grad_output[0].detach()
            return hook, grad_hook

        for name, layer in self.named_modules():
            if isinstance(layer, (nn.Conv2d, nn.Linear)):
                hook, grad_hook = hook_fn(name)
                layer.register_forward_hook(hook)
                layer.register_backward_hook(grad_hook)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

    def analyze_layer_performance(self):
        stats = defaultdict(list)
        for name, param in self.named_parameters():
            if "weight" in name or "bias" in name:
                data = param.detach().cpu().view(-1)
                grad_norm = self.gradients.get(name, torch.tensor(0.0)).norm().item()
                stats["Layer"].append(name)
                stats["Shape"].append(tuple(param.shape))
                stats["Mean"].append(data.mean().item())
                stats["Std"].append(data.std().item() if data.numel() > 1 else 0.0)
                stats["Min"].append(data.min().item())
                stats["Max"].append(data.max().item())
                stats["Grad Norm"].append(grad_norm)
                stats["Sample Values"].append(data[:5].tolist())

        df = pd.DataFrame(stats)
        # Xác định layer kém nhất (dựa trên grad norm nhỏ và std thấp)
        df["Performance Score"] = df["Std"] * df["Grad Norm"] / (df["Mean"].abs() + 1e-6)
        return df

In [39]:
# 3. Hàm huấn luyện với theo dõi loss và gradient
def train_model(root_dir, epochs=5, batch_size=32, learning_rate=1e-3, dropout=0.5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load data
    dataset = DeepfakeDataset(root_dir)
    train_size = int(0.7 * len(dataset))
    val_size = int(0.15 * len(dataset))
    test_size = len(dataset) - train_size - val_size
    train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
        dataset, [train_size, val_size, test_size]
    )

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    # Khởi tạo model
    model = CNNModel(dropout_rate=dropout).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)  # Thêm L2 regularization

    # Lưu initial parameters
    initial_params = {name: param.detach().cpu().clone() for name, param in model.named_parameters()}

    # Huấn luyện
    train_losses = []
    val_accuracies = []
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0.0
        for images, labels in tqdm(train_loader, desc=f"[Epoch {epoch+1}] Training"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()

            # Gradient clipping để tránh exploding gradient
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(train_loader)
        train_losses.append(avg_loss)

        # Validation
        model.eval()
        val_preds, val_targets = [], []
        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f"[Epoch {epoch+1}] Validating"):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                preds = torch.argmax(outputs, dim=1)
                val_preds.extend(preds.cpu().numpy())
                val_targets.extend(labels.cpu().numpy())

        acc = accuracy_score(val_targets, val_preds)
        f1 = f1_score(val_targets, val_preds)
        precision = precision_score(val_targets, val_preds)
        recall = recall_score(val_targets, val_preds)
        val_accuracies.append(acc)
        print(f"[Epoch {epoch+1}] Train Loss: {avg_loss:.4f} | Val Acc: {acc:.4f} | F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f}")

    # Test
    model.eval()
    test_preds, test_targets = [], []
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Testing"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)
            test_preds.extend(preds.cpu().numpy())
            test_targets.extend(labels.cpu().numpy())

    test_acc = accuracy_score(test_targets, test_preds)
    test_f1 = f1_score(test_targets, test_preds)
    test_precision = precision_score(test_targets, test_preds)
    test_recall = recall_score(test_targets, test_preds)
    print(f"Test Accuracy: {test_acc:.4f} | F1: {test_f1:.4f} | Precision: {test_precision:.4f} | Recall: {test_recall:.4f}")

    # Phân tích layer
    df = model.analyze_layer_performance()
    df.to_csv("weights_summary.csv", index=False)
    print("Weights summary saved to weights_summary.csv")

    # Xác định layer kém nhất
    print("\nTop 5 Layers kém nhất (dựa trên Performance Score):")
    print(df.sort_values(by="Performance Score").head(5)[["Layer", "Shape", "Mean", "Std", "Grad Norm", "Performance Score"]])

    # So sánh thay đổi weights
    final_params = {name: param.detach().cpu().clone() for name, param in model.named_parameters()}
    print("\nWeight/Bias Changes Summary:")
    changes = []
    for name in initial_params:
        if name in final_params:
            delta = final_params[name] - initial_params[name]
            mean_change = torch.mean(torch.abs(delta)).item()
            std_change = torch.std(delta).item()
            changes.append({
                "layer": name,
                "mean_change": mean_change,
                "std_change": std_change
            })
    changes.sort(key=lambda x: x["mean_change"], reverse=True)
    for ch in changes[:5]:
        print(f"{ch['layer']:<40} | Mean Δ: {ch['mean_change']:.6f} | Std Δ: {ch['std_change']:.6f}")

    # Lưu model
    torch.save(model.state_dict(), "cnn_deepfake_detector.pt")
    print("Model saved to cnn_deepfake_detector.pt")

    return model, train_losses, val_accuracies

In [32]:
# Test
model.eval()
test_preds, test_targets = [], []
with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Testing"):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)
        test_preds.extend(preds.cpu().numpy())
        test_targets.extend(labels.cpu().numpy())

test_acc = accuracy_score(test_targets, test_preds)
test_f1 = f1_score(test_targets, test_preds)
test_precision = precision_score(test_targets, test_preds)
test_recall = recall_score(test_targets, test_targets) # Fixed a potential bug here.
print(f"Test Accuracy: {test_acc:.4f} | F1: {test_f1:.4f} | Precision: {test_precision:.4f} | Recall: {test_recall:.4f}")

NameError: name 'test_loader' is not defined

In [ ]:
# 4. Phân tích và vẽ biểu đồ
def analyze_and_plot(model, train_losses, val_accuracies):
    df = pd.read_csv("weights_summary.csv")
    df["Short Layer"] = df["Layer"].apply(lambda x: x.replace(".weight", "_W").replace(".bias", "_B"))
    df_weights = df[df["Layer"].str.contains("weight")]
    df_biases = df[df["Layer"].str.contains("bias")]

    # Vẽ biểu đồ
    plt.figure(figsize=(10, 8))
    plt.barh(df_weights["Short Layer"], df_weights["Mean"], color='skyblue')
    plt.xlabel("Mean Value")
    plt.title("Mean of Weights by Layer")
    plt.gca().invert_yaxis()
    plt.grid(axis='x')
    plt.tight_layout()
    plt.savefig("weight_mean_barh.png")
    plt.close()

    plt.figure(figsize=(10, 8))
    plt.barh(df_biases["Short Layer"], df_biases["Std"], color='orange')
    plt.xlabel("Standard Deviation")
    plt.title("Std of Biases by Layer")
    plt.gca().invert_yaxis()
    plt.grid(axis='x')
    plt.tight_layout()
    plt.savefig("bias_std_barh.png")
    plt.close()

    # Vẽ histogram cho weights
    for name, param in model.named_parameters():
        if 'weight' in name:
            plt.figure(figsize=(6, 4))
            data = param.detach().cpu().numpy().flatten()
            plt.hist(data, bins=50, color='skyblue')
            plt.title(f"Weight Distribution: {name}")
            plt.grid(True)
            plt.savefig(f"weight_dist_{name}.png")
            plt.close()

    # Vẽ loss và accuracy
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.title("Training Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.grid(True)
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(val_accuracies, label='Validation Accuracy', color='orange')
    plt.title("Validation Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig("training_metrics.png")
    plt.close()

In [14]:
# 5. Đề xuất cải thiện
def suggest_improvements(df):
    print("\nĐề xuất cải thiện:")
    # Layer kém: std thấp, grad norm nhỏ -> có thể chết hoặc không học được
    weak_layers = df[df["Performance Score"] < df["Performance Score"].quantile(0.3)]
    for _, row in weak_layers.iterrows():
        layer = row["Layer"]
        std = row["Std"]
        grad_norm = row["Grad Norm"]
        print(f"\nLayer: {layer}")
        if std < 0.01:
            print("- Vấn đề: Std quá thấp, có thể layer không học được")
            print("- Giải pháp:")
            print("  + Kiểm tra requires_grad, đảm bảo = True")
            print("  + Tăng learning rate cho layer này (e.g., x2)")
            print("  + Thêm BatchNorm nếu chưa có")
        if grad_norm < 1e-3:
            print("- Vấn đề: Gradient quá nhỏ, nguy cơ vanishing gradient")
            print("  + Dùng ReLU hoặc LeakyReLU thay vì sigmoid/tanh")
            print("  + Giảm độ sâu model nếu không cần thiết")
            print("  + Thử initialization như He hoặc Xavier")
        if std > 1.0 and grad_norm > 1.0:
            print("- Vấn đề: Weight biến động lớn, có thể gây bất ổn")
            print("  + Thêm L2 regularization mạnh hơn")
            print("  + Dùng gradient clipping (max_norm ~ 1.0)")
            print("  + Giảm learning rate")

In [ ]:
##Huấn luyện mô hình

In [29]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Huấn luyện ngắn vài epoch để test
for epoch in range(3):
    for inputs, labels in dataloader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}: Loss = {loss.item():.4f}")

KeyboardInterrupt: 

In [ ]:
#In trọng số & độ lệch từng layer
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"\nLayer: {name}")
        print(f"Shape: {param.shape}")
        print(f"Mean: {param.data.mean():.6f}, Std: {param.data.std():.6f}")


# kiểm tra lớp layer kém nhất

In [ ]:
#Kiểm tra vanishing gradient
def check_gradients(model):
    for name, param in model.named_parameters():
        if param.grad is not None:
            grad_norm = param.grad.norm().item()
            print(f"{name} - Grad Norm: {grad_norm:.6f}")
        else:
            print(f"{name} - No gradient (frozen or not used)")

# Sau mỗi backward(), có thể gọi:
check_gradients(model)

NameError: name 'model' is not defined

In [ ]:
#Vẽ histogram của weight từng layer
for name, param in model.named_parameters():
    if 'weight' in name:
        plt.figure(figsize=(6, 2))
        plt.hist(param.data.cpu().numpy().flatten(), bins=50)
        plt.title(f"Weight distribution: {name}")
        plt.grid(True)
        plt.show()


In [ ]:
#Xem đầu ra sau ReLU từng layer (activation maps)
def hook_fn(module, input, output):
    print(f"{module.__class__.__name__}: Output mean={output.mean().item():.4f}, std={output.std().item():.4f}")

for layer in model.children():
    if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
        layer.register_forward_hook(hook_fn)


In [33]:
# Run the training function to get the test_loader
model, train_losses, val_accuracies = train_model(data_dir)

Scanning directory: /content/frames2/frames
Checking /content/frames2/frames/real...
Checking /content/frames2/frames/fake...
Found 42217 images


TypeError: CNNModel.__init__() got an unexpected keyword argument 'dropout_rate'